In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#boost
import lightgbm as lgb 
from sklearn.metrics import mean_squared_error


In [ ]:
data=pd.read_csv("../input/craigslist-carstrucks-data/vehicles.csv")

In [ ]:
data

In [ ]:
data.info()

In [ ]:
# visualisation of null values
data.isna().sum()
#index of columns that have over than 0.25 null values 
null_columns=data.columns[data.isna().mean()>0.25]
#axis=1 for columns axis=0 for rows  
data=data.drop(null_columns,axis=1)

In [ ]:
unneded_columns=['id','url','region_url','image_url','description','posting_date']
data=data.drop(unneded_columns,axis=1)

In [ ]:
sns.heatmap(data.corr())


In [ ]:
#serching for numb of unique values in categorical columns
{column:len(data[column].unique()) for column in data.columns if data.dtypes[column]=='object'} 

In [ ]:
data=data.drop('model',axis=1)

In [ ]:
def encodeCategories(data,columns,prefix):
    df=data.copy()
    for column,prefix in zip(columns,prefix):
        #representing categories in dummie variabes [458213,1]
        dummies=pd.get_dummies(data[column],prefix=prefix)
        df=pd.concat([df,dummies],axis=1)
        df=df.drop(column,axis=1)
    return(df)

In [ ]:
data=encodeCategories(data,['region', 'manufacturer', 'fuel','title_status','transmission', 'type', 'state'],
                      ['reg','manuf','fuel','tit_stat','transm','type','state'])
data


# Fill in all null values with mean value




In [ ]:
for column in data.columns:
    data[column]=data[column].fillna(data[column].mean())

#  Splitting and Scaling


In [ ]:
y=data.loc[:,'price']
X=data.drop('price',axis=1)
#StandardScaler for transforming values to v £[-1,1]
scaler=StandardScaler()
X=scaler.fit_transform(X)


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,random_state=34)

In [ ]:
regressor=LinearRegression()
regressor.fit(X_train,y_train)
y_pred=regressor.predict(X_test)

In [ ]:
lightModel=lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=31, n_estimators=100,reg_lambda=1.0)
lightModel.fit(X_train,y_train)
light_y_pred=lightModel.predict(X_test)

In [ ]:
lin_loss=np.sqrt(mean_squared_error(y_test,y_pred))
lightloss=np.sqrt(mean_squared_error(y_test,light_y_pred))
print("linear:",lin_loss)
print("light:",lightloss)